In [3]:
import pandas as pd

In [11]:
df=pd.read_csv("AABA.csv")

In [13]:
df.head()

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,39.69,41.22,38.79,40.91,24232729,AABA
1,2006-01-04,41.22,41.90,40.77,40.97,20553479,AABA
2,2006-01-05,40.93,41.73,40.85,41.53,12829610,AABA
3,2006-01-06,42.88,43.57,42.80,43.21,29422828,AABA
4,2006-01-09,43.10,43.66,42.82,43.42,16268338,AABA


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3019 entries, 0 to 3018
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3019 non-null   object 
 1   Open    3019 non-null   float64
 2   High    3019 non-null   float64
 3   Low     3019 non-null   float64
 4   Close   3019 non-null   float64
 5   Volume  3019 non-null   int64  
 6   Name    3019 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 165.2+ KB


In [19]:
df['Date'] = pd.to_datetime(df['Date'])

In [21]:
df = df.sort_values(by='Date', ascending=True)

In [23]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
df[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']])

In [27]:
import numpy as np

In [29]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 3])  # Predict 'Close' price
    return np.array(X), np.array(y)

seq_length = 60  # Use last 60 days as input
X, y = create_sequences(df[['Open', 'High', 'Low', 'Close', 'Volume']].values, seq_length)

In [31]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [37]:
 import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [35]:
!pip install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/11/c5/2370d96b31eb1841c3a0883a492c15278a6718ccad61bb6a649c80d1d9eb/torch-2.6.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/88/53/4ad334b9b1d8dd99836869fec139cb74a27781298360b91b9506c53f1d10/torchvision-0.21.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/1f/31/417d6955585be76842e9b0159d3801c0b5f9a4ea0db39db1a72bc262c861/torchaudio-2.6.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for sympy==1.13.1 from https://files.pythonhosted.org/packages/b2/fe/81695a1aa331a842b582453b605175f419fe8540355886031328089d840a/sympy-1.13.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB 991.0 kB/s eta 0:03:26
   -------

In [41]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [43]:
class StockPriceRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockPriceRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)  # RNN layer
        out = self.fc(out[:, -1, :])  # Take last time step's output
        return out

# Model parameters
input_size = 5   # Features: Open, High, Low, Close, Volume
hidden_size = 64  # Number of RNN hidden units
output_size = 1  # Predicting Close price

# Instantiate model
model = StockPriceRNN(input_size, hidden_size, output_size)

In [46]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")


Epoch [10/50], Loss: 0.024555
Epoch [20/50], Loss: 0.024208
Epoch [30/50], Loss: 0.024104
Epoch [40/50], Loss: 0.024102
Epoch [50/50], Loss: 0.024102


In [54]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

# Convert predictions back to NumPy array if they are PyTorch tensors
y_pred = y_pred.numpy() if isinstance(y_pred, torch.Tensor) else y_pred
y_test = y_test.numpy() if isinstance(y_test, torch.Tensor) else y_test

# Ensure y_test is 2D for proper concatenation
y_test = y_test.reshape(-1, 1)

# Reverse normalization (only for visualization)
y_pred_actual = scaler.inverse_transform(np.concatenate((np.zeros((y_pred.shape[0], 4)), y_pred), axis=1))[:, 4]
y_test_actual = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], 4)), y_test), axis=1))[:, 4]

In [58]:
# Ensure data is properly assigned before calling the function
data = df[['Open', 'High', 'Low', 'Close', 'Volume']].values  # Replace 'df' with your dataset

future_prices = predict_future(model, data, seq_length, future_days=10)
print("Predicted future stock prices:", future_prices)


Predicted future stock prices: [0.22901137 0.25637442 0.23492332 0.2502907  0.23727997 0.22872128
 0.22916473 0.23060228 0.2321314  0.23204799]


In [60]:
# Ensure data is properly assigned before calling the function
data = df[['Open', 'High', 'Low', 'Close', 'Volume']].values  # Replace 'df' with your dataset

future_prices = predict_future(model, data, seq_length, future_days=20)
print("Predicted future stock prices:", future_prices)


Predicted future stock prices: [0.22901137 0.25637442 0.23492332 0.2502907  0.23727997 0.22872128
 0.22916473 0.23060228 0.2321314  0.23204799 0.23314063 0.23297466
 0.23289822 0.23273097 0.23280002 0.23278464 0.2328126  0.23278959
 0.23280115 0.23279311]
